In [1]:
import pandas as pd
import os
from glob import glob

In [2]:
# all raw files 
raw_data_path = "../data/raw"

csv_files = glob(os.path.join(raw_data_path, "*.csv"))

if not csv_files:
    print("Не найдено CSV-файлов в указанной директории.")
else:
    print(f"Найдено {len(csv_files)} файлов. Начинаю объединение...")

    dataframes = []
    for file in csv_files:
        df = pd.read_csv(file)
        dataframes.append(df)

    combined_df = pd.concat(dataframes, ignore_index=True)

Найдено 18 файлов. Начинаю объединение...


# Инверсирование левой руки 

In [3]:

# Найдём записи левой руки
left_hands = combined_df[combined_df['is_right_hand'] == 0]

# Меняем x-координаты
for i in range(21):  # 21 точка
    combined_df.loc[combined_df['is_right_hand'] == 0, f'x{i}'] = 1.0 - combined_df[f'x{i}']

# заменить is_right_hand = 0 на 1
# combined_df.loc[combined_df['is_right_hand'] == 0, 'is_right_hand'] = 1




# Сохранение в файл
Сохранение в новый файл, в который все сохранить 


In [4]:
# new file
output_file = "../data/processed/data.csv"
combined_df.to_csv(output_file, index=False)
print(f"Готово! Итоговый файл сохранён в {output_file}. Размер: {combined_df.shape}")

Готово! Итоговый файл сохранён в ../data/processed/data.csv. Размер: (112367, 44)


# Анализ полученных данных

In [5]:
print(f"Total number of samples: {len(combined_df)}\n")


Total number of samples: 112367



In [6]:
combined_df.describe()

,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,...,y16,x17,y17,x18,y18,x19,y19,x20,y20,is_right_hand
count,112367.000000,112367.000000,112367.000000,112367.000000,112367.000000,112367.000000,112367.000000,112367.000000,112367.000000,112367.000000,...,112367.000000,112367.000000,112367.000000,112367.000000,112367.000000,112367.000000,112367.000000,112367.000000,112367.000000,112367.000000
mean,0.636070,0.570940,0.609114,0.552227,0.585298,0.529079,0.569312,0.511379,0.563737,0.498427,...,0.503617,0.609628,0.517923,0.589363,0.508306,0.589044,0.506978,0.590402,0.502272,0.489423
std,0.113209,0.155782,0.113682,0.146246,0.112852,0.144610,0.116043,0.151512,0.122169,0.162066,...,0.179126,0.131209,0.158432,0.147366,0.165377,0.150405,0.173079,0.151794,0.180728,0.499890
min,0.063948,0.059947,0.029576,0.084038,0.009065,0.084308,0.030947,0.063560,0.010394,-0.007056,...,0.003100,-0.009789,0.012573,-0.031824,0.039273,-0.025132,0.050455,-0.014215,0.020951,0.000000
25%,0.560312,0.460475,0.532469,0.448571,0.509859,0.428954,0.493522,0.406129,0.485093,0.384383,...,0.378556,0.524400,0.407136,0.487354,0.392080,0.482482,0.380768,0.482064,0.369351,0.000000
50%,0.640422,0.561101,0.618120,0.545015,0.595211,0.518850,0.579951,0.495602,0.574794,0.480106,...,0.492704,0.601344,0.502711,0.577183,0.490045,0.576768,0.488100,0.576642,0.484643,0.000000
75%,0.711483,0.673292,0.686653,0.647701,0.662788,0.615896,0.647333,0.598714,0.644793,0.592096,...,0.616993,0.701481,0.609580,0.699430,0.604765,0.702561,0.611909,0.705569,0.612927,1.000000
max,1.038216,1.125709,1.024293,1.070749,0.955869,1.000152,0.967850,1.009102,0.981843,1.037755,...,1.084793,1.008961,1.011888,1.018961,1.034550,1.023803,1.060576,1.036301,1.079373,1.000000


In [7]:
gesture_counts = combined_df['gesture'].value_counts()
print("Gesture distribution:")
print(gesture_counts, "\n")


Gesture distribution:
gesture
sheriff     25847
mute        22486
mafia       20912
if          20817
question     5805
don          5000
civilian     4500
last         3500
cool         3500
Name: count, dtype: int64 

